In [18]:
import asyncio
import nest_asyncio
import pandas as pd
from pyppeteer import launch
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [19]:
# Permite asyncio en Jupyter Notebook
nest_asyncio.apply()

In [20]:
# Función para obtener las fechas de ayer y hoy en formato adecuado
def get_dates():
    today = datetime.now().strftime("%b %d, %Y")
    yesterday = (datetime.now() - timedelta(days=1)).strftime("%b %d, %Y")
    return yesterday, today

In [21]:
 # Función para obtener el HTML renderizado con Pyppeteer
async def fetch_calendar():
    url = "https://www.investing.com/economic-calendar/"
    browser = await launch(headless=True)
    page = await browser.newPage()
    await page.goto(url, {"waitUntil": "networkidle2"})
    await asyncio.sleep(5)  # Espera para cargar la página completamente
    content = await page.content()
    await browser.close()
    return content   

In [22]:
# Función para analizar el HTML y extraer los datos del calendario económico
def parse_calendar(html, dates):
    soup = BeautifulSoup(html, "html.parser")
    events = []
    
    for row in soup.select(".js-event-item"):
        date = row.select_one(".theDay").text.strip() if row.select_one(".theDay") else ""
        time = row.select_one(".time") or ""
        currency = row.select_one(".left.flagCur span") or ""
        event_name = row.select_one(".event") or ""
        actual = row.select_one(".act") or ""
        forecast = row.select_one(".fore") or ""
        previous = row.select_one(".prev") or ""
        
        if date in dates:
            events.append({
                "Fecha": date,
                "Hora": time.text.strip() if time else "",
                "Divisa": currency.text.strip() if currency else "",
                "Evento": event_name.text.strip() if event_name else "",
                "Actual": actual.text.strip() if actual else "",
                "Pronóstico": forecast.text.strip() if forecast else "",
                "Previo": previous.text.strip() if previous else ""
            })

    return pd.DataFrame(events)

In [23]:
# Ejecutar el código
if __name__ == "__main__":
    dates = get_dates()
    html_content = asyncio.run(fetch_calendar())
    df = parse_calendar(html_content, dates)
    print(df)

[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.
